<a href="https://colab.research.google.com/github/tejinderpa/LANGCHAIN/blob/main/Retrieval_Augmented_Generation_Using_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [2]:
!pip install -q youtube-transcript-api langchain-community faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [3]:
!pip install langchain-google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [22]:
video_id = "88bMVbx1dzM"

In [26]:
from youtube_transcript_api import YouTubeTranscriptApi

ytt_api = YouTubeTranscriptApi()
transcript_list = ytt_api.fetch(video_id)

In [33]:
# Extract and combine the text from the transcript list
# Accessing the text attribute using dot notation
transcript_text = " ".join([d.text for d in transcript_list])
print("Full Transcript Text:")
print(transcript_text)

Full Transcript Text:
This is a tiny piece of metal 
just three millimeters across. And here's what happens if 
you just keep zooming in: 1,000 times, 100,000 times, 50 million times. Each of these blobs is an actual atom. I saw this the other day at 
the University of Sydney, and it kind of blew my mind 
because up until just 30 years ago,   directly seeing atoms like this 
was thought to be impossible. The rooms that you're going to see here are 
perhaps the most shielded rooms on campus, or even in the whole of Sydney, I would say. And perhaps also the most expensive. That's wild. So why is it so hard to see atoms? Well, you can't actually see 
atoms with visible light. That's because while light has wavelengths 
between 380 and 750 nanometers, an atom is still over 3000 times 
smaller, just 0.1 nanometers. And if the wavelength of light is much 
bigger than the thing you're trying to see, the light will just diffract or bend 
around it so you won't be able to see it. So if you want

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Use the split_text method on the combined transcript_text string
chunks = splitter.split_text(transcript_text)

# Optional: If you want to create LangChain Document objects from the chunks
# from langchain_core.documents import Document
# chunks_as_documents = [Document(page_content=chunk) for chunk in chunks]

print(f"Split the transcript into {len(chunks)} chunks.")
# print(chunks[:5]) # Display the first few chunks to verify

Split the transcript into 25 chunks.


In [38]:
chunks[24]

"aberration correction make? If you want to see atoms and if you want to,\xa0\nfor example, measure in atomic distances,\xa0\xa0 and if you want to learn what type of atoms\xa0\nyou have, you need aberration correction. Any research that's material science.\xa0\nMaterials engineering, chemical engineering... You need to see what's happening at the atomic\xa0\xa0level because you want to relate\xa0\nthe properties to the structure. If you can't see the structure at the atomic\xa0\nlevel, you only have half of the information. So that was a game changer. That's why nowadays every university in\xa0\nprinciple needs a microscope like that."

In [44]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document # Import Document

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

# Convert string chunks to Document objects
chunks_as_documents = [Document(page_content=chunk) for chunk in chunks]

vectorindex_google = FAISS.from_documents(chunks_as_documents, embeddings)

In [45]:
vectorindex_google.index_to_docstore_id

{0: 'e42831b3-c6d7-49ad-a21d-a9890d37d23a',
 1: 'dce4a31e-d547-4194-bff1-3746daf9a12f',
 2: 'b1bb928a-a54d-4978-b7a1-346ebbb36451',
 3: '09a139b7-4aa1-4a32-af31-e05b13a7085b',
 4: '6bd18750-1d1c-47f3-a2e1-433a3d4048ca',
 5: '35f2f4c2-1614-4b53-90ce-8cc9b170ea96',
 6: 'dcb1bf79-b903-4668-81c5-89354057740f',
 7: 'fbca9b4e-a065-4032-b457-4a7f628284a7',
 8: '84c1e0ad-393c-46ce-ba2f-7976c7c293ae',
 9: '5da2295f-5549-486b-a545-17998ad7a8cd',
 10: '542cb9d8-d679-456f-bc39-a327d54d26bd',
 11: 'e8785b0f-0cf1-44f2-a53e-b98a020573ec',
 12: '0dfbcb67-6c4f-49c2-8251-1aa9a4da6f71',
 13: '97cdba99-ef9b-4f28-a032-3db1e3c0e660',
 14: 'bdeb8924-977b-4fdd-8d90-7aea98ee6d72',
 15: 'c8e58fd5-95d9-49dd-882b-7e1c89c3f72b',
 16: '4b1820e8-4986-410c-9393-e040a4413d2a',
 17: '8398cd6c-be72-41dc-88ab-558956662cbd',
 18: '0d3d55f2-d650-4937-9bfd-e827fcfb836b',
 19: '8b5edcd7-52dd-4967-bac4-52ba79fb5520',
 20: '99740c5c-638b-4a29-9193-fe1018b16d71',
 21: 'f4dc5735-4e8c-4b7f-a55f-fa9198af0b0e',
 22: '646bed4b-34fd-

In [46]:
retriever = vectorindex_google.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [47]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b259ca52810>, search_kwargs={'k': 4})

In [48]:
retriever.invoke("What is deepmind")

[Document(id='2b595ed1-01ac-489e-a2da-7e83216cd04a', metadata={}, page_content="aberration correction make? If you want to see atoms and if you want to,\xa0\nfor example, measure in atomic distances,\xa0\xa0 and if you want to learn what type of atoms\xa0\nyou have, you need aberration correction. Any research that's material science.\xa0\nMaterials engineering, chemical engineering... You need to see what's happening at the atomic\xa0\xa0level because you want to relate\xa0\nthe properties to the structure. If you can't see the structure at the atomic\xa0\nlevel, you only have half of the information. So that was a game changer. That's why nowadays every university in\xa0\nprinciple needs a microscope like that."),
 Document(id='e8785b0f-0cf1-44f2-a53e-b98a020573ec', metadata={}, page_content="when the gas atoms hit the needle,\xa0\xa0 they got ionized and were ejected\xa0\noff perpendicular to the surface. And that could form an impression\xa0\nof the atomic structure of the tip. But

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)


In [51]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [52]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [53]:
retrieved_docs

[Document(id='2b595ed1-01ac-489e-a2da-7e83216cd04a', metadata={}, page_content="aberration correction make? If you want to see atoms and if you want to,\xa0\nfor example, measure in atomic distances,\xa0\xa0 and if you want to learn what type of atoms\xa0\nyou have, you need aberration correction. Any research that's material science.\xa0\nMaterials engineering, chemical engineering... You need to see what's happening at the atomic\xa0\xa0level because you want to relate\xa0\nthe properties to the structure. If you can't see the structure at the atomic\xa0\nlevel, you only have half of the information. So that was a game changer. That's why nowadays every university in\xa0\nprinciple needs a microscope like that."),
 Document(id='8b5edcd7-52dd-4967-bac4-52ba79fb5520', metadata={}, page_content='beam through a second hexapole , one that worked\xa0\xa0 the opposite way, so it would unbend the\xa0\ndistorted image back into a circular shape. But now they, calculated this\xa0\nnew image mi

In [54]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"aberration correction make? If you want to see atoms and if you want to,\xa0\nfor example, measure in atomic distances,\xa0\xa0 and if you want to learn what type of atoms\xa0\nyou have, you need aberration correction. Any research that's material science.\xa0\nMaterials engineering, chemical engineering... You need to see what's happening at the atomic\xa0\xa0level because you want to relate\xa0\nthe properties to the structure. If you can't see the structure at the atomic\xa0\nlevel, you only have half of the information. So that was a game changer. That's why nowadays every university in\xa0\nprinciple needs a microscope like that.\n\nbeam through a second hexapole , one that worked\xa0\xa0 the opposite way, so it would unbend the\xa0\ndistorted image back into a circular shape. But now they, calculated this\xa0\nnew image might have the remnants\xa0\xa0 of that tiny divergence still in its center, with\xa0\nspherical aberration pointing in the opposite way. So if they got their ma

In [55]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [56]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      aberration correction make? If you want to see atoms and if you want to,\xa0\nfor example, measure in atomic distances,\xa0\xa0 and if you want to learn what type of atoms\xa0\nyou have, you need aberration correction. Any research that's material science.\xa0\nMaterials engineering, chemical engineering... You need to see what's happening at the atomic\xa0\xa0level because you want to relate\xa0\nthe properties to the structure. If you can't see the structure at the atomic\xa0\nlevel, you only have half of the information. So that was a game changer. That's why nowadays every university in\xa0\nprinciple needs a microscope like that.\n\nbeam through a second hexapole , one that worked\xa0\xa0 the opposite way, so it would unbend the\xa0\ndistorted image back into a circular shape. But now they, ca

#Step-4-Generation

In [57]:
answer = llm.invoke(final_prompt)
print(answer.content)

No, the provided text does not discuss nuclear fusion.


#Building a chain

In [58]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [59]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [60]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [65]:
parallel_chain.invoke('Why it is hard to see atoms')

{'context': 'This is a tiny piece of metal\xa0\njust three millimeters across. And here\'s what happens if\xa0\nyou just keep zooming in: 1,000 times, 100,000 times, 50 million times. Each of these blobs is an actual atom. I saw this the other day at\xa0\nthe University of Sydney, and it kind of blew my mind\xa0\nbecause up until just 30 years ago,\xa0\xa0 directly seeing atoms like this\xa0\nwas thought to be impossible. The rooms that you\'re going to see here are\xa0\nperhaps the most shielded rooms on campus, or even in the whole of Sydney, I would say. And perhaps also the most expensive. That\'s wild. So why is it so hard to see atoms? Well, you can\'t actually see\xa0\natoms with visible light. That\'s because while light has wavelengths\xa0\nbetween 380 and 750 nanometers, an atom is still over 3000 times\xa0\nsmaller, just 0.1 nanometers. And if the wavelength of light is much\xa0\nbigger than the thing you\'re trying to see, the light will just diffract or bend\n\naberration 

In [66]:
parser = StrOutputParser()

In [67]:
main_chain = parallel_chain | prompt | llm | parser

In [69]:
main_chain.invoke("Why it is hard to see atoms")

"It is hard to see atoms because visible light has wavelengths between 380 and 750 nanometers, while an atom is over 3000 times smaller (0.1 nanometers).  If the wavelength of light is much bigger than what you're trying to see, the light will diffract or bend around it, making it impossible to see."